# Using Machine Learning to Control for Covariates in a Regression
This notebook will illustrate machine learning methods for controlling for a large set of covariates in a regression estimating the effect of elite college attendance on later-life earnings. There are two basic approaches. The first is "Post-Double Selection Lasso" (Belloni, Chernozhukov, Hansen). The second is "Double-Debiased Machine Learning" (Chernozhukov, Chetverikov, Demirer, Duflo, Hansen, Newey, Robins)

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')
%matplotlib inline
%cd '/content/gdrive/My Drive/Econ 484/datasets'

Mounted at /content/gdrive
/content/gdrive/My Drive/Econ 484/datasets


## Load useful packages: 
pandas, numpy, linear_model (from sklearn), and KFold (from sklearn.model_selection)

Try it yourself first

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold

### Cheat

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold

## Read in data and have a look at the head and shape
college.csv is in the "datasets" subfolder

Try it yourself

In [3]:
college = pd.read_csv('college.csv')
print(college.head())

  inst  hstopten  hsmiss  ath  ...    v27_x_24    v27_x_25    v27_x_26    v27_x_27
0  MIA         0       0    1  ...  101.935000   90.250000  10193.5000   90.250000
1  PSU         0       1    0  ...  105.460800  103.225590  10546.0800  103.225590
2  PSU         0       0    0  ...    0.000000    0.000000      0.0000    0.000000
3  MIA         1       0    0  ...   98.394096   84.088898   9839.4102   84.088898
4  MIA         0       0    0  ...  108.265690  101.808110  10826.5700  101.808110

[5 rows x 409 columns]


### Cheat

In [ ]:
collegedata=pd.read_csv('/content/gdrive/My Drive/Econ 484/datasets/college.csv')
print(collegedata.head())
print("Shape: {}".format(str(collegedata.shape)))

  inst  hstopten  hsmiss  ath  ...    v27_x_24    v27_x_25    v27_x_26    v27_x_27
0  MIA         0       0    1  ...  101.935000   90.250000  10193.5000   90.250000
1  PSU         0       1    0  ...  105.460800  103.225590  10546.0800  103.225590
2  PSU         0       0    0  ...    0.000000    0.000000      0.0000    0.000000
3  MIA         1       0    0  ...   98.394096   84.088898   9839.4102   84.088898
4  MIA         0       0    0  ...  108.265690  101.808110  10826.5700  101.808110

[5 rows x 409 columns]
Shape: (14238, 409)


## Define outcome, regressor of interest, covariate matrix and sampling weights
y = lowninc 

d = matsat2 

X = everything except y, d, inst, and instwt

sampling weights = instwt

Try it yourself:

In [5]:
y=college['lowninc']
d=college['matsat2']
X=college.loc[:,[x for x in college.columns if x not in ('lowninc','matsat2','inst','instwt')]]
weights = college['instwt']

##if you don't want to deal with reshaping series as numpy arrays then use loc to select the variables

### Cheat

In [ ]:
y=collegedata.loc[:,'lowninc']
d=collegedata.loc[:,['matsat2']]
X=collegedata.drop(['lowninc','matsat2','inst','instwt'],axis=1)
instwt=collegedata.loc[:,'instwt']

## Simple Regression with no Controls
Regress y on d and print out coefficient
Try it yourself

In [ ]:
y

0        10.596635
1        10.596635
2        11.042922
3        10.596635
4        11.042922
           ...    
14233    11.736069
14234    11.379395
14235    12.072541
14236     9.615806
14237    11.042922
Name: lowninc, Length: 14238, dtype: float64

In [ ]:

reg = linear_model.LinearRegression().fit(d.to_numpy().reshape(-1,1),y.to_numpy().reshape(-1,1),weights.to_numpy().reshape(-1))
print(reg.coef_)

[[0.10948733]]


### Cheat

In [ ]:
lm=linear_model.LinearRegression()
lm.fit(d,y,instwt)
print("Simple regression effect of selective college: {:.3f}".format(lm.coef_[0]))

Simple regression effect of selective college: 0.109


## Post Double Selection Lasso

### Step 1: Lasso the outcome on X
Try it yourself

In [ ]:
lasso1 = linear_model.Lasso().fit(X,y)
print(lasso1.coef_)

[ 0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.521e+00, tolerance: 8.149e-01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


#### Cheat

In [ ]:
lassoy = linear_model.Lasso(alpha=0.001, max_iter=1000,normalize=True).fit(X, y)


### Step 2: Lasso the treatment on X
Try it yourself

In [ ]:
lasso2 = linear_model.Lasso().fit(X,d)
print(lasso2.coef_)

[ 0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.746e+02, tolerance: 1.225e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


#### Cheat

In [ ]:
lassod = linear_model.Lasso(alpha=0.001, max_iter=1000,normalize=True).fit(X, d)

### Step 3: Form the union of controls
Try it yourself

In [ ]:
Xunion=X.iloc[:,(lasso2.coef_!=0) + (lasso1.coef_!=0)]
Xunion.head()

,v26_x_1,v26_x_2,v26_x_3,v26_x_4,v26_x_6,v26_x_8,v26_x_9,v26_x_10,v26_x_11,v26_x_12,v26_x_13,v26_x_14,v26_x_16,v26_x_17,v26_x_18,v26_x_19,v26_x_21,v26_x_22,v26_x_23,v26_x_24,v26_x_25,v26_x_26,v27_x_26
0,9967.2236,6438.0,0,1073,0,0,0,1073,1073,0,0,0,0,1073,0,0,2146,11513.289,10193.5000,11513.289,10193.5000,1151329,10193.5000
1,10601.3500,9342.0,1038,0,0,0,1038,0,1038,0,0,0,0,1038,0,0,2076,10774.440,10546.0800,10774.440,10546.0800,1077444,10546.0800
2,10221.9470,11937.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1038,10774.440,0.0000,10774.440,0.0000,1077444,0.0000
3,11157.0550,13197.9,1073,0,0,1073,0,0,1073,0,0,0,0,1073,0,0,2146,11513.289,9839.4102,11513.289,9839.4102,1151329,9839.4102
4,11157.4920,12661.4,0,0,0,0,0,0,1073,0,0,0,0,1073,0,0,2146,11513.289,10826.5700,11513.289,10826.5700,1151329,10826.5700


#### Cheat

In [ ]:
Xunion=X.iloc[:,(lassod.coef_!=0) + (lassoy.coef_!=0)]
Xunion.head()

,female,satacchigh,v3_x_3,v24_x_1,v24_x_22
0,0,10.73,0,99.672234,115.13289
1,1,10.38,1,106.013500,107.74440
2,0,10.38,0,102.219470,107.74440
3,1,10.73,1,111.570540,115.13289
4,0,10.73,0,111.574910,115.13289


### Concatenate treatment with union of controls and regress y on that and print out estimate
Try yourself

In [ ]:
controls = pd.concat([d,Xunion],axis=1)
post_double = linear_model.LinearRegression().fit(controls,y,weights)
print(post_double.coef_[0])

-0.0016203999915255759


#### Cheat

In [ ]:
rhs=pd.concat([d,Xunion],axis=1)
fullreg=linear_model.LinearRegression().fit(rhs,y,instwt)
print("PDS regression effect of selective college: {:.3f}".format(fullreg.coef_[0]))

NameError: ignored

## Double-Debiased Machine Learning
For simplicity, we will first do it without sample splitting

### Step 1: Ridge outcome on Xs, get residuals
Try yourself

In [6]:
ridge1 = linear_model.Ridge().fit(X,y)
yerror=y-ridge1.predict(X)


#### Cheat

In [ ]:
ridgey = linear_model.Ridge(alpha=0.001, max_iter=1000,normalize=True).fit(X, y)
yresid=y-ridgey.predict(X)

### Step 2: Ridge treatment on Xs, get residuals
Try yourself

In [7]:
ridge2 = linear_model.Ridge().fit(X,d)
derror=d-ridge2.predict(X)

#### Cheat

In [ ]:
ridged = linear_model.Ridge(alpha=0.001, max_iter=1000,normalize=True).fit(X, d)
dresid=d-ridged.predict(X)

### Step 3: Regress y resids on d resids and print out estimate
Try yourself

In [8]:
double_debiased = linear_model.LinearRegression().fit(derror.to_numpy().reshape(-1,1),yerror,weights)
print(double_debiased.coef_)

[-0.01104281]


####Cheat

In [ ]:
ddmlreg=linear_model.LinearRegression().fit(dresid,yresid,instwt)
print("DDML regression effect of selective college: {:.3f}".format(ddmlreg.coef_[0]))

DDML regression effect of selective college: -0.008


### The real thing: with sample splitting

In [9]:
#cross validation would come before this step if we were doing the full ML process
# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)

# initialize array to hold each fold's regression coefficient
coeffs=np.zeros(5)

# Now loop through each fold
ii=0
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index], d.iloc[test_index]
  wt_train, wt_test = weights.iloc[train_index], weights.iloc[test_index]
  # Do DDML thing
  # Ridge y on training folds:
  ridge1.fit(X_train, y_train)

  # but get residuals in test set
  yresid=y_test-ridge1.predict(X_test)
  
  #Ridge d on training folds
  ridge2.fit(X_train, d_train)

  #but get residuals in test set
  dresid=d_test-ridge2.predict(X_test)

  # regress resids on resids
  ddmlreg=linear_model.LinearRegression().fit(dresid.to_numpy().reshape(-1,1),yresid,wt_test)

  # save coefficient in a vector
  coeffs[ii]=ddmlreg.coef_[0]
  ii+=1

# Take average
print("Double-Debiased Machine Learning effect of selective college: {:.3f}".format(np.mean(coeffs)))
coeffs

Double-Debiased Machine Learning effect of selective college: -0.013


array([ 0.05008222, -0.0102456 , -0.04505427,  0.0111317 , -0.07024997])

In [ ]:
list(kf.split(X))

[(array([    0,     1,     2, ..., 14234, 14235, 14237]),
  array([    8,    14,    15, ..., 14221, 14223, 14236])),
 (array([    1,     2,     4, ..., 14233, 14235, 14236]),
  array([    0,     3,    10, ..., 14231, 14234, 14237])),
 (array([    0,     1,     2, ..., 14235, 14236, 14237]),
  array([   26,    28,    30, ..., 14230, 14232, 14233])),
 (array([    0,     1,     3, ..., 14235, 14236, 14237]),
  array([    2,     6,     7, ..., 14225, 14226, 14229])),
 (array([    0,     2,     3, ..., 14234, 14236, 14237]),
  array([    1,     4,     5, ..., 14219, 14222, 14235]))]

## Now do DDML using Random Forest!

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor()
rf2 = RandomForestRegressor()


# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)

# initialize array to hold each fold's regression coefficient
coeffs=np.zeros(5)

# Now loop through each fold
ii=0
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index], d.iloc[test_index]
  wt_train, wt_test = weights.iloc[train_index], weights.iloc[test_index]
  # Do DDML thing
  # Ridge y on training folds:
  rf1.fit(X_train, y_train)

  # but get residuals in test set
  yresid=y_test-rf1.predict(X_test)
  
  #Ridge d on training folds
  rf2.fit(X_train, d_train)

  #but get residuals in test set
  dresid=d_test-rf2.predict(X_test)

  # regress resids on resids
  ddmlreg=linear_model.LinearRegression().fit(dresid.to_numpy().reshape(-1,1),yresid,wt_test)

  # save coefficient in a vector
  coeffs[ii]=ddmlreg.coef_[0]
  ii+=1

# Take average
print("Double-Debiased Machine Learning effect of selective college: {:.3f}".format(np.mean(coeffs)))
coeffs

Double-Debiased Machine Learning effect of selective college: -0.008


array([ 0.0674927 , -0.00311223, -0.04754356, -0.02154066, -0.03608414])